In [324]:
import pandas as pd
import numpy as np
from functools import reduce
import math



In [325]:
def read_file(file_name, sheet_name):
    return pd.read_excel(file_name, sheet_name)  

In [326]:
folder_name ="C:/Users/iaw/Desktop/Second Training Task/Input/"
fund_investment_summary_file_name = folder_name +"CEPRES_Platform_Fund_CEPRES_Example_Portfolio_Investment_Summary_USD_($)_Net.xlsx"
mapping_ids_file_name = folder_name +"mapping_Ids.xlsx"

In [327]:
def load_investment_summary_file(fund_investment_summary_file_name, sheet_name):
    investment_sumary_df = read_file(fund_investment_summary_file_name,sheet_name)
    investment_sumary_df = investment_sumary_df.T

    new_coulmns = list(investment_sumary_df.iloc[0].values)
    investment_sumary_df.columns = new_coulmns
    investment_sumary_df = investment_sumary_df.iloc[1:, :].reset_index(drop=True)
    investment_sumary_df.dropna(axis=1, how="all", inplace=True)
    return investment_sumary_df

In [328]:
def load_mapping_Ids(mapping_ids_file_name,sheet_name):
    dfs = read_file(mapping_ids_file_name,sheet_name)
    new_header = dfs.iloc[0].str.replace('\n','') #grab the first row for the header
    dfs = dfs[1:] #take the data less the header row
    dfs.columns = new_header #set the header row as the df header
    country_region_mapping = dict(zip( dfs["Country"],dfs["Major Market"]))
    return country_region_mapping

In [329]:
def generate_performance_attribution_table(investment_sumary_df,country_region_mapping):
   
    investment_sumary_df ["Region"]= investment_sumary_df["Geographical Focus"].apply(lambda x : country_region_mapping[x])
    funds = investment_sumary_df.groupby("Region")["IRR [% p.a.] - Net of Fees"].size().apply(lambda x : (x/investment_sumary_df["Geographical Focus"].size)*100 ).reset_index(name ='Funds')
    mean = investment_sumary_df.groupby("Region")["IRR [% p.a.] - Net of Fees"].apply(np.mean).reset_index(name ='Mean')
    median = investment_sumary_df.groupby("Region")["IRR [% p.a.] - Net of Fees"].apply(np.median).reset_index(name="Median")
    data_frames = [funds, mean,median]
    result = reduce(lambda  left,right: pd.merge(left,right,on=['Region'],
                                                how='outer'), data_frames)
    last_row = {"Region":"All","Funds":result["Funds"].sum(),"Mean":investment_sumary_df["IRR [% p.a.] - Net of Fees"].mean(),"Median":investment_sumary_df["IRR [% p.a.] - Net of Fees"].median()}
    result = result.append(last_row,ignore_index=True).round(2)
    result["Funds"] = result["Funds"].apply(lambda x : str(round(x ,2)) + "%")
    result["Mean"] = result["Mean"].apply(convert_to_percentage)
    result["Median"] = result["Median"].apply(convert_to_percentage)
    print(result)



In [330]:
def generate_exposure_geographical_focuse_table(investment_sumary_df,country_region_mapping):
    
    investment_sumary_df ["Region"]= investment_sumary_df["Geographical Focus"].apply(lambda x : country_region_mapping[x])
    funds = investment_sumary_df.groupby("Region")["IRR [% p.a.] - Net of Fees"].size().apply(lambda x : (x/investment_sumary_df["Geographical Focus"].size)*100 ).reset_index(name ='Funds')
    mean = investment_sumary_df.groupby("Region")["Absolute Commitment"].apply(np.mean).reset_index(name ='Mean')
    median = investment_sumary_df.groupby("Region")["Absolute Commitment"].apply(np.median).reset_index(name="Median")
    min = investment_sumary_df.groupby("Region")["Absolute Commitment"].min().reset_index(name="Min")
    max = investment_sumary_df.groupby("Region")["Absolute Commitment"].max().reset_index(name="Max")
    
    data_frames = [funds, mean,median,min,max]
    result = reduce(lambda  left,right: pd.merge(left,right,on=['Region'],
                                                how='outer'), data_frames)
                
    last_row = {"Region":"All","Funds":result["Funds"].sum(),"Mean":investment_sumary_df["Absolute Commitment"].mean(),
    "Median":investment_sumary_df["Absolute Commitment"].median(),"Min":investment_sumary_df["Absolute Commitment"].min(),
    "Max":investment_sumary_df["Absolute Commitment"].max()}

    result = result.append(last_row,ignore_index=True).round(2)
    result ["Funds"] = result["Funds"].apply(lambda x : str(round(x ,2)) + "%")

    print(result)










In [331]:
def convert_to_percentage(x) -> str: 
    if  isinstance(x, str):
        return x
    elif not math.isnan(x):
        percentange = str(round(x *100 ,2)) + "%"
    else:
        percentange = ""
    return percentange

In [332]:
investment_sumary_df = load_investment_summary_file(fund_investment_summary_file_name,"Fund Investment Summary")
country_region_mapping = load_mapping_Ids(mapping_ids_file_name,"Country")
print("\t\t\t\tCEPRES Example GP \n\t Fund Net IRR by Geographic Focus in USD ($) by % Invested Capital\n")
generate_performance_attribution_table(investment_sumary_df,country_region_mapping)
print("\n\n\t\t\t\tCEPRES Example GP \n\t\t Fund Net Information in USD ($) by % Invested Capital\n")
generate_exposure_geographical_focuse_table(investment_sumary_df,country_region_mapping)

				CEPRES Example GP 
	 Fund Net IRR by Geographic Focus in USD ($) by % Invested Capital

             Region   Funds   Mean Median
0              Asia   7.41%  0.18%  0.18%
1            Europe  25.93%  0.15%  0.15%
2  Northern America  66.67%  0.17%  0.18%
3               All  100.0%  0.16%  0.16%


				CEPRES Example GP 
		 Fund Net Information in USD ($) by % Invested Capital

             Region   Funds   Mean  Median    Min     Max
0              Asia   7.41%  29.50   29.50  24.00   35.00
1            Europe  25.93%  61.53   50.00  29.33  158.71
2  Northern America  66.67%  48.18   43.34  20.00  100.00
3               All  100.0%  50.26   40.01  20.00  158.71
